Create Player Ratings

In [1]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats_v2.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
28536,202324_WestCoast_Adelaide,West Coast,Ryan Maric,202324,Ryan_Maric,KEY_FORWARD,CD_I1029416,18,196,93,41,RIGHT,VIC,2023.0,2023.0,Drouin Vic)/Gippsland U18/Box Hill Hawks (VFL),1.0,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2004-09-06,71,0,0.0,NaN,0,0,NaN,NaN,NaN,0,NaN,2,NaN,NaN,NaN,NaN,77.8,9,49,NaN,NaN,NaN,0,2,0.0,1,0,NaN,3,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,NaN,6,NaN,5,0,NaN,114.0,0,NaN,NaN,1,NaN,3,NaN,0,NaN,0,2,0,0,10,2,8,NaN,NaN,Half Forward Flank Left,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,0.000000,-0.058709,0.040843,0.052497,-0.135987,0.188484,2.130433,0.966152,1.164281,0.291146,0.000000,0.403961,4.395202,Ryan Maric_2023,0,0.000000,West Coast,Adelaide,Adelaide,24,24,202324
28537,202324_WestCoast_Adelaide,West Coast,Sam Petrevski-Seton,202324,Sam_Petrevski-Seton,MEDIUM_FORWARD,CD_I994539,25,181,78,10,RIGHT,WA,2016.0,2017.0,Halls Creek (WA)/Claremont (WAFL)/Carlton,6.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-02-19,61,0,0.0,NaN,0,3,NaN,NaN,NaN,0,NaN,6,NaN,NaN,NaN,NaN,83.3,18,64,NaN,NaN,NaN,0,1,0.0,1,0,NaN,12,0,NaN,NaN,NaN,0,NaN,4,NaN,NaN,NaN,6,NaN,3,0,NaN,87.0,1,NaN,NaN,2,NaN,2,NaN,0,NaN,0,3,0,0,18,4,12,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023,0.000000,-0.273480,0.031157,-0.401388,-0.277125,-0.124263,-2.362353,-1.703999,-0.658353,-0.161335,0.113615,-0.044036,0.159084,Sam Petrevski-Seton_2023,0,0.000000,West Coast,Adelaide,Adelaide,24,24,202324
28538,202324_WestCoast_Adelaide,West Coast,Shannon Hurn,202324,Shannon_Hurn,MEDIUM_DEFENDER,CD_I240283,35,187,95,25,RIGHT,SA,2005.0,2006.0,Angaston (SA)/Central District (SANFL),13.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1987-09-04,83,0,0.0,NaN,0,2,NaN,NaN,NaN,0,NaN,2,NaN,NaN,NaN,NaN,95.5,22,89,NaN,NaN,NaN,0,1,0.0,0,0,NaN,4,0,NaN,NaN,NaN,0,NaN,4,NaN,NaN,NaN,18,NaN,6,0,NaN,506.0,3,NaN,NaN,7,NaN,2,NaN,0,NaN,1,2,0,1,12,2,10,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.000000,-0.335596,0.032884,0.695787,0.144239,0.551549,1.323515,2.124795,-0.801279,-1.550106,0.177283,-0.459052,-1.629679,Shannon Hurn_2023,0,0.000000,West Coast,Adelaide,Adelaide,24,24,202324
28539,202324_WestCoast_Adelaide,West Coast,Tim Kelly,202324,Tim_Kelly,MIDFIELDER,CD_I295898,29,184,83,11,RIGHT,WA,2017.0,2018.0,Palmyra (WA)/South Fremantle (WAFL)/Geelong,24.0,nationalDraft,https://s.afl.com.au/stat

In [3]:
player_stats[['Position_Group', 'Position_Sub_Group', 'Position']].value_counts().sort_index()

Position_Group  Position_Sub_Group  Position                
Back            Back-Pocket         Back Pocket Left            1207
                                    Back Pocket Right           1196
                Half-Back           Half Back Flank Left        1195
                                    Half Back Flank Right       1186
                Key-Back            Centre Half Back            1187
                                    Full Back                   1209
Forward         Forward-Pocket      Forward Pocket Left         1190
                                    Forward Pocket Right        1196
                Half-Forward        Half Forward Flank Left     1183
                                    Half Forward Flank Right    1171
                Key-Forward         Centre Half Forward         1182
                                    Full Forward                1219
Interchange     Interchange         Interchange                 4763
                                    Substi

Different Positions are expected to do different things

In [4]:
match = player_stats[player_stats['Match_ID'] == "202323_WesternBulldogs_WestCoast"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
28081,202323_WesternBulldogs_WestCoast,West Coast,Alex Witherden,202323,Alex_Witherden,MEDIUM_DEFENDER,CD_I998128,24,188,85,23,RIGHT,VIC,2016.0,2017.0,"St Joseph&apos;s, Geel (Vic)/Geel College (Vic...",23.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-09-10,93,0,1.0,0.0,0,3,0.0,0.0,0.0,0,44.4,8,0.0,0.0,0.0,7.0,90.9,22,73,20.0,11.0,0.0,0,3,0.0,0,0,3.0,9,0,0.0,0.0,0.0,0,0.0,8,84.6,6.0,6.0,13,1.4,3,0,0.0,377.0,5,7.0,11.8,8,0.0,3,2.0,0,3.0,0,1,0,0,18,4,10,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.000000,-0.267964,0.023892,0.487451,0.280099,0.207352,1.037687,1.599591,-0.561904,-1.010768,0.023430,-0.087556,-0.381674,Alex Witherden_2023,0,0.000000,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
28082,202323_WesternBulldogs_WestCoast,West Coast,Andrew Gaff,202323,Andrew_Gaff,MIDFIELDER,CD_I290801,31,184,83,3,LEFT,VIC,2010.0,2011.0,Kew (Vic)/Carey Grammar (Vic)/Oakleigh U18,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1992-06-16,86,0,1.0,6.0,0,2,0.0,0.0,0.0,0,22.7,5,0.0,0.0,0.0,16.0,78.3,23,99,18.0,8.0,1.0,0,0,0.0,0,0,5.0,12,0,0.0,0.0,0.0,3,0.0,3,72.7,0.0,0.0,11,0.9,6,0,0.0,218.0,0,21.0,5.1,3,0.0,4,0.0,1,0.0,1,6,0,1,22,3,17,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023,2.148563,0.191014,0.052414,-0.063434,0.129387,-0.192821,0.607441,1.700765,-1.093324,-0.230022,0.138102,0.589806,3.155958,Andrew Gaff_2023,0,-2.148563,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
28083,202323_WesternBulldogs_WestCoast,West Coast,Bailey J. Williams,202323,Bailey_J._Williams,RUCK,CD_I1006114,23,201,101,32,RIGHT,VIC,2018.0,2020.0,Pines (Vic)/Western Port SC (Vic)/Dandenong U18,35.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-04-17,81,0,0.0,26.0,3,4,0.0,0.0,0.0,0,90.9,10,0.0,0.0,0.0,6.0,50.0,10,74,5.0,3.0,1.0,2,2,0.0,0,0,6.0,4,41,9.0,22.0,41.8,2,0.0,1,50.0,0.0,0.0,6,1.5,1,0,0.0,126.0,2,14.0,8.7,0,98.0,3,3.0,0,1.0,2,2,0,5,11,4,1,NaN,NaN,Ruck,FINAL_TEAM,Ruck,Ruck,NaN,NaN,2023,0.000000,0.251066,0.067486,0.613062,0.577126,0.035936,4.382892,4.323413,0.059479,-0.014674,0.022624,0.359049,3.785106,Bailey J. Williams_2023,0,0.000000,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
28084,202323_WesternBulldogs_WestCoast,West Coast,Campbell Chesser,202323,Campbell_Chesser,MIDFIELDER,CD_I1017057,20,186,85,18,RIGHT,NSW,2021.0,2023.0,L

Team Ratings - Combination of VAEP and Expected VAEP Values should be a correct order (Season by Season)

In [5]:
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,1.080668,0.318364,0.762304,6.534905,3.036248,3.498657,-0.461340,0.190552,0.545439,1.933806,Brandon Starcevich_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.458517,0.623663,-0.165146,2.267625,3.311832,-1.044207,-0.487945,-0.039690,0.266572,1.916089,Callum Ah Chee_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.734292,0.734427,-0.000135,6.061680,6.251205,-0.189524,0.197894,-0.035491,1.216250,5.936939,Charlie Cameron_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncra

In [6]:
sorted(list(set(player_stats['Round_ID'])))[-10:]

['202315',
 '202316',
 '202317',
 '202318',
 '202319',
 '202320',
 '202321',
 '202322',
 '202323',
 '202324']

In [7]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby('Team').sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby('Team').count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary


In [8]:
team_summary = create_team_summary(player_stats, rounds = None)
team_summary

,Games,Score_F,xScore_F,vaep_value_F,vaep_value_received_F,offensive_value_F,defensive_value_F,exp_vaep_value_F,exp_vaep_value_received_F,exp_offensive_value_F,exp_defensive_value_F,Score_A,xScore_A,vaep_value_A,vaep_value_received_A,offensive_value_A,defensive_value_A,exp_vaep_value_A,exp_vaep_value_received_A,exp_offensive_value_A,exp_defensive_value_A,Score_pergame_F,Score_pergame_A,Score_diff,Score_pergame_diff,xScore_pergame_F,xScore_pergame_A,xScore_diff,xScore_pergame_diff,vaep_value_pergame_F,vaep_value_pergame_A,vaep_value_diff,vaep_value_pergame_diff,vaep_value_received_pergame_F,vaep_value_received_pergame_A,vaep_value_received_diff,vaep_value_received_pergame_diff,offensive_value_pergame_F,offensive_value_pergame_A,offensive_value_diff,offensive_value_pergame_diff,defensive_value_pergame_F,defensive_value_pergame_A,defensive_value_diff,defensive_value_pergame_diff,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_diff,exp_vaep_value_pergame_diff,exp_vaep_value_received_pergame_F,exp_vaep_value_received_pergame_A,exp_vaep_value_received_diff,exp_vaep_value_received_pergame_diff,exp_offensive_value_pergame_F,exp_offensive_value_pergame_A,exp_offensive_value_diff,exp_offensive_value_pergame_diff,exp_defensive_value_pergame_F,exp_defensive_value_pergame_A,exp_defensive_value_diff,exp_defensive_value_pergame_diff
Adelaide,66,5297,5468.778525,968.050033,1030.117118,878.079854,89.970179,5930.147523,6141.636719,5784.255238,145.892282,5583,5605.439277,1010.170031,1063.387891,932.679873,77.490158,6050.920730,6448.937626,5985.778873,65.141855,80.257576,84.590909,-286,-4.333333,82.860281,84.930898,-136.660752,-2.070617,14.667425,15.305607,-42.119997,-0.638182,15.607835,16.111938,-33.270773,-0.504103,13.304240,14.131513,-54.600018,-0.827273,1.363185,1.174093,12.480021,0.189091,89.850720,91.680617,-120.773207,-1.829897,93.055102,97.711176,-307.300908,-4.656074,87.640231,90.693619,-201.523635,-3.053388,2.210489,0.986998,80.750427,1.223491
Brisbane Lions,72,6618,6623.561327,1203.256893,1172.182948,1089.482523,113.774370,7150.422622,7145.468762,6810.100102,340.322519,5448,5466.721025,1043.816354,1113.825607,960.292842,83.523511,6118.735131,6795.059133,6056.722043,62.013090,91.916667,75.666667,1170,16.250000,91.993907,75.926681,1156.840303,16.067226,16.711901,14.497449,159.440539,2.214452,16.280319,15.469800,58.357341,0.810519,15.131702,13.337401,129.189680,1.794301,1.580200,1.160049,30.250859,0.420151,99.311425,84.982432,1031.687491,14.328993,99.242622,94.375821,350.409629,4.866800,94.584724,84.121139,753.378059,10.463584,4.726702,0.861293,278.309430,3.865409
Carlton,67,5365,5658.283588,1011.959527,1097.459411,915.403604,96.555923,6295.306497,6634.402020,6066.012552,229.293950,5210,5311.150363,998.580003,1050.122970,888.255594,110.324409,5984.228919,6342.569224,5714.142306,270.086621,80.074627,77.761194,155,2.313433,84.451994,79.270901,347.133225,5.181093,15.103874,14.904179,13.379524,0.199694,16.379991,15.673477,47.336440,0.706514,13.662740,13.257546,27.148010,0.405194,1.441133,1.646633,-13.768486,-0.205500,93.959798,89.316850,311.077578,4.642949,99.020926,94.665212,291.832796,4.355713,90.537501,85.285706,351.870245,5.251795,3.422298,4.031144,-40.792671,-0.608846
Collingwood,70,5594,5651.166428,1033.863362,1098.222320,952.510437,81.352924,6214.093982,6693.826664,6102.285982,111.807991,5333,5425.548742,1003.502349,1048.749300,898.929873,104.572476,6017.679578,6369.937559,5724.400330,293.279244,79.914286,76.185714,261,3.728571,80.730949,77.507839,225.617686,3.223110,14.769477,14.335748,30.361012,0.433729,15.688890,14.982133,49.473019,0.706757,13.607292,12.841855,53.580564,0.765437,1.162185,1.493893,-23.219552,-0.331708,88.772771,85.966851,196.414404,2.805920,95.626095,90.999108,323.889104,4.626987,87.175514,81.777148,377.885652,5.398366,1.597257,4.189703,-181.471252,-2.592446
Essendon,68,5413,5509.383946,1006.944016,1059.030644,913.424649,93.519367,6074.098425,6378.898846,5882.399476,191.698952,5852,5832.266638,1078.2

In [9]:
def zscale_stats(summary_data, stats):
    
    for col in stats:
        summary_data[col+'_z'] = (summary_data[col] - summary_data[col].mean()) / np.std(summary_data[col])
    summary_data = summary_data.fillna(0)
    
    return summary_data

In [10]:
# rating_factors = ['Score_pergame_diff', 'xScore_pergame_diff', 'vaep_value_pergame_diff', 'exp_vaep_value_pergame_diff']
rating_factors = ['xScore_pergame_diff', 'exp_vaep_value_pergame_diff']
team_ratings = team_summary[rating_factors]
team_ratings = zscale_stats(team_ratings, rating_factors)
team_ratings['team_rating'] = team_ratings[[x+"_z" for x in rating_factors]].mean(axis=1)
team_ratings.sort_values(by = 'team_rating', ascending = False)

,xScore_pergame_diff,exp_vaep_value_pergame_diff,xScore_pergame_diff_z,exp_vaep_value_pergame_diff_z,team_rating
Melbourne,19.601527,18.336274,1.365828,1.374259,1.370044
Geelong,15.630550,14.862880,1.093438,1.117947,1.105692
Brisbane Lions,16.067226,14.328993,1.123392,1.078549,1.100971
Western Bulldogs,11.966825,11.052701,0.842123,0.836781,0.839452
Port Adelaide,10.629083,10.635970,0.750360,0.806029,0.778195
Sydney,8.255865,7.541470,0.587569,0.577676,0.582623
Carlton,5.181093,4.642949,0.376654,0.363785,0.370220
Collingwood,3.223110,2.805920,0.242345,0.228225,0.235285
Richmond,2.988707,2.652933,0.226266,0.216936,0.221601
Fremantle,0.613026,0.484792,0.063306,0.056942,0.060124


In [11]:
def create_team_ratings(team_summary, rating_factors = []):
    
    team_ratings = team_summary[rating_factors]
    team_ratings = zscale_stats(team_ratings, rating_factors)
    team_ratings['team_rating'] = team_ratings[[x+"_z" for x in rating_factors]].mean(axis=1)
    
    return team_ratings.sort_values(by = 'team_rating', ascending = False)

In [12]:
player_stats_2023 = player_stats[player_stats['Season'] == 2023]
team_summary = create_team_summary(player_stats_2023, rounds = None)
team_ratings = create_team_ratings(team_summary, rating_factors = ['Score_pergame_diff', 'xScore_pergame_diff', 'vaep_value_pergame_diff', 'exp_vaep_value_pergame_diff'])
team_ratings

,Score_pergame_diff,xScore_pergame_diff,vaep_value_pergame_diff,exp_vaep_value_pergame_diff,Score_pergame_diff_z,xScore_pergame_diff_z,vaep_value_pergame_diff_z,exp_vaep_value_pergame_diff_z,team_rating
Brisbane Lions,17.521739,20.636576,2.124218,18.607771,0.973699,1.222237,0.824454,1.188757,1.052287
Melbourne,17.478261,13.588956,2.806179,13.095933,0.971282,0.804829,1.089138,0.836633,0.925471
Collingwood,18.869565,11.507281,2.995925,10.119384,1.048598,0.681539,1.162782,0.646476,0.884849
Adelaide,14.565217,12.541449,1.965159,12.184946,0.809402,0.742789,0.762720,0.778435,0.773337
Port Adelaide,10.304348,16.312639,1.193166,15.950805,0.572622,0.966144,0.463093,1.019016,0.755219
Carlton,9.521739,8.742249,1.327817,8.018763,0.529131,0.517775,0.515354,0.512278,0.518635
Geelong,10.217391,5.525846,1.515633,5.503092,0.567789,0.327278,0.588249,0.351565,0.458720
Sydney,8.260870,6.673633,1.082584,5.874215,0.459064,0.395257,0.420174,0.375274,0.412442
St Kilda,5.260870,6.191527,0.932388,5.677035,0.292351,0.366704,0.361880,0.362677,0.345903
Western Bulldogs,6.608696,5.203363,0.904917,4.921555,0.367251,0.308178,0.351217,0.314413,0.335265


In [13]:
def get_cumulative_round_summary(player_stats, current_round_id, rating_factor, num_rounds = None):
    
    current_season = int(current_round_id[:4])
    if "F" in current_round_id:
        current_round_id_int = int(finals_round_map[current_round_id[4:]])
        current_round_id_str = finals_round_map[current_round_id[4:]]
    else:
        current_round_id_int = int(current_round_id[4:])
        current_round_id_str = current_round_id[4:]
    current_round_id_num = int(str(current_season) + str(current_round_id_str))

    past_seasons_player_stats = player_stats[(player_stats['Season'] < current_season)]
    current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round_id_int)]
    rating_player_stats = pd.concat([past_seasons_player_stats, current_round_player_stats], axis=0)

    round_summary_dict = {}
    for round_id in list(set(rating_player_stats['Round_ID'])):
        season = int(round_id[:4])
        if "F" in round_id:
            round_id_int = int(finals_round_map[round_id[4:]])
            round_id_str = finals_round_map[round_id[4:]]
        else:
            round_id_int = int(round_id[4:])
            round_id_str = round_id[4:]
        seasons_player_stats = rating_player_stats[(rating_player_stats['Season'] < season)]
        round_player_stats = rating_player_stats[(rating_player_stats['Season'] == season) & (rating_player_stats['Round'] <= round_id_int)]
        latest_round_player_stats = pd.concat([seasons_player_stats, round_player_stats], axis=0)
        
        latest_round_team_summary = create_team_summary(latest_round_player_stats, rounds = num_rounds)
        
        round_summary_dict[int(str(season) + str(round_id_str))] = latest_round_team_summary[rating_factor]
        
    round_summary_df = pd.DataFrame.from_dict(round_summary_dict)
    # round_summary_df.T.sort_index()
    round_summary_df = round_summary_df.reindex(sorted(round_summary_df.columns), axis=1).sort_values(by=int(current_round_id_num), ascending=False)
    return round_summary_df

In [14]:
xscore_f_summary = get_cumulative_round_summary(player_stats, current_round_id='202323', rating_factor = 'xScore_pergame_F', num_rounds = 10)
# xscore_f_summary[[x for x in list(xscore_f_summary) if '2023' in str(x)]]
xscore_f_summary

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,84.197005,80.916275,94.170831,96.014360,93.691386,92.292777,94.283868,92.814433,89.829921,88.161757,87.109941,87.897614,84.255133,82.211379,83.213148,81.958491,77.824745,79.736297,81.566776,82.265521,80.437207,78.765945,78.648312,78.147995,76.239568,76.558390,77.813672,78.813835,77.337026,75.340948,78.758755,82.438127,79.202410,86.063949,88.956862,90.245759,91.121945,86.708798,88.168966,88.960490,88.780163,87.554606,92.523238,91.133704,85.118728,85.698579,82.582922,84.542328,84.024997,84.761493,85.723001,86.248734,83.870981,78.728247,79.476105,77.220654,80.020944,82.947058,79.640493,79.326902,87.645736,85.101119,82.682512,80.950374,81.640782,76.907157,76.404238,78.988193,81.150256,83.314718,79.006424,84.089353,91.579918,93.635301,93.178419,94.388845,95.620636
Sydney,92.468489,96.490469,102.182044,100.628085,95.712307,86.909251,85.298589,82.317101,83.265167,81.540361,80.717459,80.438926,73.983516,71.540994,70.882156,77.890592,79.667315,81.484403,82.895318,85.080151,82.337013,81.248963,88.371843,88.694392,90.758253,88.856797,88.461322,91.668544,88.478048,84.456936,86.939191,89.173243,90.319563,89.318182,86.615880,88.193400,86.663419,85.603699,85.840392,88.985678,88.226266,86.307723,83.309017,86.548525,87.633269,88.850810,93.799860,97.231192,97.384158,97.617142,98.233830,99.559127,99.106501,91.211632,94.132920,95.032392,91.792972,87.844194,90.919471,84.666628,87.579582,85.896753,83.801368,88.212852,84.757298,80.856257,77.203333,75.058459,83.511043,90.129111,86.172077,87.509136,89.017708,89.453630,89.816930,90.276809,93.270885
Brisbane Lions,69.728083,76.918707,77.877148,71.201923,78.109112,80.580829,83.019726,85.769847,88.494462,89.130849,93.661811,91.932190,93.280875,97.650889,96.650696,96.558666,94.348315,92.964967,95.600101,94.211471,94.321791,99.776944,101.780185,98.812394,97.518297,98.119091,100.663984,100.050570,93.155466,100.662080,94.289952,91.524391,88.661559,92.896551,93.951944,92.690686,92.831567,94.961683,95.449535,91.028838,93.914979,89.513752,90.050373,89.147433,90.052309,90.711345,91.845605,91.029563,91.531172,87.403967,87.993514,90.361435,86.051300,85.992340,80.539173,80.273236,75.574758,76.544266,84.184163,88.714113,90.026768,91.277134,96.025534,96.851100,100.191759,101.753231,103.023790,102.741593,94.860025,97.162897,98.730982,98.810089,94.648051,91.310045,90.350962,90.294019,92.532492
Adelaide,NaN,108.378634,99.162397,98.131887,90.098748,91.609992,88.482220,81.910835,82.124786,82.253137,82.445489,79.407239,78.303426,76.329267,77.366000,74.788185,70.733286,72.140082,72.283369,70.412942,67.509906,65.991860,70.661352,70.661352,70.077779,69.452515,74.969722,79.566462,76.310992,79.279103,85.479533,87.913017,82.335251,79.562066,77.389196,76.337936,77.601022,75.607084,77.661299,76.928692,75.555070,78.161808,74.910962,74.212102,75.682634,76.165626,76.824121,78.368501,79.536171,77.393178,76.782496,73.580899,75.776848,78.967455,82.702821,85.221361,86.509384,88.877042,90.790629,94.926008,92.019632,89.023677,90.826187,87.141269,85.749490,85.438132,89.338281,89.165855,86.188439,91.081282,90.936579,90.286817,88.793229,95.075961,97.640794,99.412688,92.204167
Greater Western Sydney,92.188832,82.721049,76.868707,74.746656,77.132635,74.200382,80.657708,82.861826,84.300674,85.986476,83.254111,84.365316,86.776829,88.495435,86.324811,87.025669,81.635208,78.961565,77.470639,73.659019,74.543875,76.684448,79.473349,77.451727,77.636462,78.946420,79.953783,82.878038,81.736882,84.243810,81.207246,76.

In [15]:
xscore_a_summary = get_cumulative_round_summary(player_stats, current_round_id='202323', rating_factor = 'xScore_pergame_A', num_rounds = 10).sort_values(by=202323, ascending=True)
xscore_a_summary[[x for x in list(xscore_a_summary) if '2023' in str(x)]]

,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,79.771067,81.703919,77.671763,75.450737,80.259048,82.948715,77.053212,78.559869,77.278996,78.431750,77.522278,78.726952,79.737046,77.727323,74.373480,71.119878,73.315729,73.278618,72.665467,73.391364,71.101399,68.265446,68.850166
Greater Western Sydney,92.341902,90.304829,85.360394,94.846260,96.362520,95.098530,96.152420,93.059723,94.528854,94.950301,92.768774,94.179875,92.432792,86.432244,88.004902,85.074210,82.207477,81.184509,74.927045,73.038702,72.645079,72.494159,69.909033
St Kilda,81.540533,80.826323,80.909635,76.510405,73.461684,70.187291,72.033168,71.129227,74.920009,74.195949,78.081692,80.982978,79.022387,80.423401,82.214212,79.652598,77.094236,78.562087,73.765786,75.075582,69.924863,70.666635,70.887670
Brisbane Lions,91.608441,89.181519,85.096605,85.832793,84.577866,83.681742,81.594661,79.253326,75.694917,74.464878,69.436025,69.807376,71.424082,69.183054,70.036686,67.135750,63.581061,64.614032,64.448330,68.377362,67.211581,70.512372,71.526782
Melbourne,73.054997,72.275213,76.524568,75.143025,78.522636,81.313043,77.670349,78.256225,74.817222,77.436716,77.809127,73.685939,71.124953,71.246452,68.084884,63.661411,65.976802,66.785496,71.729804,69.938187,71.159706,72.370132,71.773154
Western Bulldogs,78.941552,77.230408,72.952820,73.014992,77.647570,78.560288,76.501663,76.013110,74.604282,72.541554,70.183265,70.563301,74.255206,73.950197,72.925941,74.377558,76.122389,77.254787,75.980414,79.790788,79.437895,79.018421,77.253497
Sydney,73.041837,72.664146,79.609620,77.583914,81.779642,84.969852,86.341754,85.750015,87.532822,84.938547,85.550894,87.403707,84.604247,88.453712,82.445222,78.247574,80.400577,80.018905,76.337439,77.131474,78.882999,78.952395,78.421891
Fremantle,74.658469,73.948526,73.318907,74.219090,79.283186,84.389571,87.730875,84.259263,82.577724,82.532562,83.951831,86.531893,85.366078,86.801093,84.795079,80.624691,79.921443,84.651256,86.159828,85.313907,84.488645,80.914380,81.596847
Adelaide,94.329530,90.593858,92.603015,92.092489,87.371520,82.355433,79.209497,80.102288,79.665021,82.195224,79.932683,80.988155,77.915530,78.653281,81.678417,79.488047,84.268438,82.898618,85.548788,81.795707,79.125506,77.999887,81.672035
Port Adelaide,76.272266,79.002517,83.112389,78.028720,77.838523,79.259196,80.021439,78.317231,75.603193,75.768984,77.593035,76.492415,73.344237,71.877585,72.961891,74.247700,72.197501,77.380543,77.681647,81.377257,84.885754,82.642373,82.043426


In [17]:
get_cumulative_round_summary(player_stats, current_round_id='202324', rating_factor = 'exp_vaep_value_pergame_diff', num_rounds=10)

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323,202324
Brisbane Lions,-20.803766,-11.002809,-4.246795,-11.346259,1.543034,2.276422,6.784241,9.235617,14.866765,16.226922,22.572250,19.882134,21.061790,29.039712,27.337916,28.696438,23.428852,20.348796,19.199855,16.482728,18.229269,26.227254,25.696993,19.250182,16.328810,16.124227,19.836896,22.085092,15.475922,25.876992,15.553073,11.832752,10.603571,15.017493,19.938454,18.554267,16.098305,18.087766,17.659344,11.366986,15.321427,8.141987,8.963989,6.074158,4.898797,4.465984,5.391183,3.624251,6.430382,1.299737,2.040523,5.417363,0.441862,2.269688,-10.697334,-9.136759,-9.719435,-9.899979,-2.193651,3.109581,6.445911,10.110618,17.826215,19.993927,27.811028,28.942184,28.470222,30.773100,22.762230,26.874146,31.461005,30.864464,27.216889,20.467841,21.052321,17.912930,19.084457,19.662865
Sydney,20.803766,7.610823,15.523308,15.807376,10.194456,-0.893037,-5.655192,-4.692900,-0.918117,-3.272661,-4.497886,-3.116444,-9.180081,-12.051154,-12.091474,0.305034,4.685931,4.622733,4.811457,7.685031,3.542123,4.277269,13.097915,14.679516,17.712807,13.022149,14.139606,14.212415,9.282255,3.795106,7.991184,13.395194,12.351291,6.792172,3.674011,8.388043,5.513882,6.561989,8.222261,13.248227,11.480593,9.821961,6.940718,10.745275,14.542607,13.345821,22.869071,24.644546,26.691411,25.838046,27.831952,26.730059,26.270062,17.168945,20.307463,21.327635,12.352685,10.217317,8.846398,0.015036,1.218331,0.333188,-3.505912,3.039805,-1.508387,-6.621629,-7.396243,-12.838422,-0.610858,10.162191,4.641033,6.244620,11.114143,10.720782,9.687055,10.617792,13.315275,16.806524
Carlton,-37.541139,-25.032283,1.499568,11.987478,9.508679,6.933339,8.161223,4.640619,1.839924,2.066193,4.965155,6.761462,1.450149,-3.378354,-1.024470,-1.253109,-3.027501,3.585044,5.204055,3.795644,3.293816,-4.409002,-7.490628,-8.322328,-11.959924,-12.525469,-14.539548,-17.289693,-18.625576,-16.285962,-17.638894,-5.340516,-4.268177,5.125603,9.967821,13.011012,13.745268,8.432998,10.926646,11.560825,13.213890,13.532734,18.755633,18.424176,10.077745,10.099671,5.406527,6.457831,5.969279,6.549164,7.977736,7.378186,5.830089,-2.949092,-0.350592,-3.626354,2.768206,6.790994,-0.732107,-4.267638,8.776662,5.648067,4.611362,2.105243,3.792837,-1.601918,-2.952835,1.461838,6.464964,11.981888,5.828526,10.032522,17.537059,18.944958,20.498221,24.102455,24.439368,16.401176
Adelaide,NaN,5.582119,6.854152,12.341035,5.531187,3.091059,-4.438009,-11.378492,-10.976303,-11.138656,-10.988000,-10.127332,-9.237117,-12.853997,-13.507340,-14.785184,-17.037503,-15.561773,-13.713508,-16.342686,-15.659993,-20.522311,-16.464772,-16.464772,-15.925564,-15.604922,-7.813373,-1.339663,-7.966071,-3.630494,-1.515305,1.280805,-2.205583,-6.283169,-10.980691,-10.591811,-7.120225,-8.923751,-1.746499,-0.819818,-4.768943,-0.985261,-9.088386,-8.223109,-2.926744,-6.021018,-6.269832,-4.299471,-7.571816,-12.453355,-10.910523,-14.931556,-11.307453,-9.365163,-11.123845,-5.405094,-5.892286,-2.529912,2.891742,12.016892,12.560107,8.368259,10.565253,4.906517,5.698575,4.446460,10.985594,10.097919,4.766482,11.360530,6.663436,8.035807,4.216576,12.830326,17.520314,20.173588,10.291662,15.920113
Greater Western Sydney,11.833229,-12.984975,-19.688331,-12.905593,-7.873030,-13.463523,-5.528255,-3.554080,-2.037291,0.483166,-4.965108,-1.316433,3.494506,2.668141,-0.983845,3.506984,-0.850145,-2.154535,-2.179509,-6.516302,-2.197942,0.324892,2.827076,-0.064849,-1.036102,-1.040278,-1.528961,-1.592669,-5.470097,-1.074223,-5.036519,-12.570422,-18.605999,

In [18]:
def get_cumulative_round_team_ratings(player_stats, current_round_id, rating_factors, num_rounds = None):
    
    current_season = int(current_round_id[:4])
    if "F" in current_round_id:
        current_round_id_int = int(finals_round_map[current_round_id[4:]])
        current_round_id_str = finals_round_map[current_round_id[4:]]
    else:
        current_round_id_int = int(current_round_id[4:])
        current_round_id_str = current_round_id[4:]
    current_round_id_num = int(str(current_season) + str(current_round_id_str))

    past_seasons_player_stats = player_stats[(player_stats['Season'] < current_season)]
    current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round_id_int)]
    rating_player_stats = pd.concat([past_seasons_player_stats, current_round_player_stats], axis=0)

    round_summary_dict = {}
    for round_id in list(set(rating_player_stats['Round_ID'])):
        season = int(round_id[:4])
        if "F" in round_id:
            round_id_int = int(finals_round_map[round_id[4:]])
            round_id_str = finals_round_map[round_id[4:]]
        else:
            round_id_int = int(round_id[4:])
            round_id_str = round_id[4:]
        seasons_player_stats = rating_player_stats[(rating_player_stats['Season'] < season)]
        round_player_stats = rating_player_stats[(rating_player_stats['Season'] == season) & (rating_player_stats['Round'] <= round_id_int)]
        latest_round_player_stats = pd.concat([seasons_player_stats, round_player_stats], axis=0)
        
        latest_round_team_summary = create_team_summary(latest_round_player_stats, rounds = num_rounds)
        latest_round_team_ratings = create_team_ratings(latest_round_team_summary, rating_factors = rating_factors)
        round_summary_dict[int(str(season) + str(round_id_str))] = latest_round_team_ratings['team_rating']
        
    round_summary_df = pd.DataFrame.from_dict(round_summary_dict)
    round_summary_df = round_summary_df.reindex(sorted(round_summary_df.columns), axis=1).sort_values(by=int(current_round_id_num), ascending=False)
    return round_summary_df

In [19]:
get_cumulative_round_team_ratings(player_stats, current_round_id='202324', rating_factors = ['xScore_pergame_diff', 'exp_vaep_value_pergame_diff'], num_rounds=10)

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323,202324
Brisbane Lions,-0.831805,-0.484859,-0.155161,-0.528269,0.080524,0.118330,0.385279,0.540114,0.937637,0.978302,1.375974,1.230718,1.472982,1.920173,2.126909,2.275011,1.830134,1.730127,1.649694,1.346019,1.458118,1.940419,1.752086,1.445821,1.218360,1.194703,1.291113,1.502566,1.167749,1.855271,1.252782,0.817311,0.661085,0.839903,1.050288,0.947456,0.842800,0.889003,0.808979,0.558081,0.723461,0.426160,0.440394,0.327166,0.285305,0.252058,0.304853,0.214946,0.367505,0.038613,0.117952,0.329322,0.076545,0.245231,-0.560638,-0.462542,-0.588222,-0.519854,-0.001516,0.361696,0.519608,0.759878,1.129160,1.316821,1.726597,1.612102,1.464711,1.542648,1.075402,1.234571,1.452078,1.430779,1.281211,1.079466,1.111275,0.948481,1.079723,1.124529
Sydney,0.831805,0.331402,0.647374,0.765368,0.473175,-0.041474,-0.291065,-0.258326,-0.047928,-0.186661,-0.255104,-0.151574,-0.629040,-0.775008,-0.911092,0.043668,0.385007,0.425820,0.444296,0.655565,0.318249,0.346447,0.940386,1.141001,1.329975,0.999251,0.963556,1.002193,0.763870,0.379580,0.658222,0.868133,0.682121,0.365042,0.189770,0.391085,0.243330,0.279071,0.321350,0.562702,0.484263,0.443609,0.305089,0.529300,0.791422,0.795210,1.403109,1.596309,1.691768,1.481574,1.653054,1.528699,1.617990,1.158761,1.326827,1.467686,0.972379,0.722843,0.672656,0.057383,0.123258,0.047276,-0.206561,0.196335,-0.068818,-0.359657,-0.379602,-0.626425,0.008778,0.477805,0.226476,0.301543,0.530642,0.572825,0.518282,0.556820,0.760473,0.970641
Carlton,-1.456199,-1.130607,0.084103,0.596142,0.438412,0.323872,0.439215,0.259974,0.114634,0.131491,0.299405,0.421281,0.090062,-0.222908,-0.091772,-0.101320,-0.248618,0.287383,0.410959,0.269851,0.241836,-0.356461,-0.517976,-0.578670,-0.774260,-0.758783,-0.740569,-0.940275,-1.118581,-0.960659,-1.180052,-0.247980,-0.155077,0.310083,0.529308,0.634066,0.687412,0.402918,0.473335,0.524894,0.592531,0.654085,0.878833,0.943707,0.581227,0.652337,0.359104,0.447492,0.400196,0.404499,0.522503,0.471522,0.423771,-0.061107,0.070454,-0.150720,0.280652,0.528303,0.040876,-0.220706,0.664056,0.432678,0.301108,0.143489,0.233316,-0.099454,-0.169291,0.067678,0.299492,0.525622,0.251827,0.458645,0.813879,0.975673,1.071049,1.260824,1.377334,0.942170
Greater Western Sydney,0.456746,-0.606684,-0.818551,-0.608277,-0.363988,-0.641527,-0.289263,-0.190005,-0.122877,0.039363,-0.294813,-0.064802,0.226282,0.167186,-0.065819,0.279942,-0.075397,-0.206143,-0.189029,-0.539637,-0.174767,0.021583,0.194838,0.044308,-0.007513,0.021814,0.029271,0.021041,-0.274170,0.012106,-0.306791,-0.719346,-0.934913,-0.540625,-0.572502,-0.694758,-0.439050,-0.577691,-0.490691,-0.470173,-0.432022,-0.443457,-0.278332,-0.607172,-0.674315,-0.753197,-1.504637,-1.110712,-1.050580,-1.257729,-1.191481,-1.049828,-1.412992,-1.454227,-1.012658,-0.657809,-0.148777,-0.750952,-0.423409,-0.430415,-0.487431,-0.441178,-0.566785,-0.739034,-0.671160,-0.681793,-0.355610,0.011741,-0.103742,-0.134961,0.040649,0.087007,0.513779,0.777378,0.768932,0.697040,1.018632,0.910643
Adelaide,NaN,0.293521,0.332854,0.641045,0.294105,0.185195,-0.209513,-0.621081,-0.656733,-0.640877,-0.632259,-0.572305,-0.635960,-0.831957,-1.019912,-1.146827,-1.301593,-1.302451,-1.150642,-1.309651,-1.243010,-1.514589,-1.103033,-1.145183,-1.062215,-0.979971,-0.370433,-0.008338,-0.473366,-0.178590,-0.074637,0.125982,-0.071756,-0.284364,-0.518453,-0.513566,-0.369510,-0.426679,-0.092820,-0.060059,-0.238227,-0.074926,-0.426976,-0.428335,-0.180236,-0.365465,-0.379506,-0.268996,-0.473528,-